In [4]:
# our own implementation of naive bayes 

#import all the required libraries that will be needed throughout the code 
from sklearn import datasets
from sklearn import model_selection
import string
import csv 
import math
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# a program to fetch all the commonwords from the csv file and store them in a list and then return the list to us.The csv of the common word is attached. Also we have converted all the words to lower case so that case sensitivity is taken care of    
def commonwords():
    li=list()
    with open("Commonwords.csv",'r') as obj1:
        obj_data=csv.reader(obj1)
        for i in obj_data:
            if len(i)!=0:
                k=i[0].split()
                for z in k:
                    li.append(str.lower(z))

    return li

#the trainer program accepts the input sentences and then converts them into words and cound the occourance of each word in each class and then returns a final dictionary that consists of the  target values as level 1 and then in each class we have the total of frequency of each word as a count 
def trainer(x_train,y_train):
    master={}
    #we make a master dictionary and then call the commonwords function to get the list of commonwords with us 
    li=commonwords()
    #now we will iterate over each and every training point 
    for i in range(len(x_train)):
        master[y_train[i]]=master.get(y_train[i],{})
        #now we will first look for its class and in taking that class of the dictionary as the key we will iterate and first of all we will convert all the sentences into words and remove out all the punctuations from them 
        j=(x_train[i].translate(str.maketrans('','', string.punctuation)).split())
        #now for each and every word we will check that if that word is not a part of common words we will then add it for that particular class and increase the frequency accordingly 
        for k in j:
            k=str.lower(k)
            if k not in li:
                master[y_train[i]][k]=master[y_train[i]].get(k,0)+1
    master2={}
    #now we will filter out the words based on a certain lower cut which means that words below that frequency would be ignored 
    for i in master:
        master2[i]=master2.get(i,{})
        for j in master[i]:
            if master[i][j]>5:
                master2[i][j]=master2[i].get(j,master[i][j])
    
    #now we will return the filtered dictionary and the commonwords list along with that as well 
    return master2,li

#now we will use this function to find out the total number of words that we have in our training data after filtering and store that in the dictionary
def total_adder(master2):
    for i in master2:
        sum=0
        for j in master2[i]:
            sum+=master2[i][j]
        master2[i]["total"]=sum
    return master2

#now we will pass the test value data to another function along with that will filter them out in the same way as we did for the training data i.e. by making the dicitonary of each test value with important words and their frequency and will remove the punctuations and then we will append those dictionaries into the new list and return the list  
def test_value_filter(x_test,li):
    test_val=list()
    for i in range(len(x_test)):
        d=dict()
        j=(x_test[i].translate(str.maketrans('','', string.punctuation)).split())
        for k in j:
            k=str.lower(k)
            if k not in li:
                d[k]=d.get(k,0)+1
        test_val.append(d)
    return test_val

# it is used to find the probablity of each class and then store it in a dictionary and return it 
def prob_finder(y_train):
    y1=dict()
    for i in y_train:
        y1[i]=y1.get(i,0)+1
    t=len(y_train)
    y_prob=dict()
    for i in y1:
        y_prob[i]=math.log(y1[i]/t)
    return y_prob

#the main  predicter function is used to find out and predict the classes from it thus we will pass and iterate over the each value and then compare it with each class of the list and we will allot the class with the highest probabllity and predict that for the particular iteration of that test case  
def predicter(master2,test_val,y_train):
    my_pred=list()
    #making call to know class wise probablity and get that in a dictionary 
    y_prob=prob_finder(y_train)
    
    for i in test_val:
        #for each iteratrion in test_val we will check on every possible classes and then will update them with the maximum one here we will be working on logarthmic probablities.Also since we will be skipping the 0 cases thus there will not be the need of La place correction as 0 probablity values are simply skipped 
        yp=-1
        prob=0
        for j in master2:
            prob2=0
            for k in i:
                if k in master2[j]:
                    prob2+=i[k]*(math.log(master2[j][k]/master2[j]["total"]))
            prob2+=y_prob[j]
            if prob2<=prob:
                prob=prob2
                yp=j
        #now after making a decision we will append them in an array and return that array 
        my_pred.append(yp)
    
    return my_pred


            
            
# we will make call to the 20newsgroup dataset in sklearn and remove the header footer  from it 
news = datasets.fetch_20newsgroups(remove=("headers","footers")) 
x = news.data
y=news.target
#now we will split the dataset into training and testing data 
x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y, test_size = 0.05, random_state = 5)
# we will make the call to the training fuction to get the list of commonwords and to get our trained data dictionary 
master2,li=trainer(x_train,y_train)
#now we will make call to total adder to add the total values 
master2=total_adder(master2)
#nwe make a call and filter the text value as the same way as we do for training dat 
test_val=test_value_filter(x_test,li)
#now we will make a call to predict function to make the predictions and get the results 
my_pred=predicter(master2,test_val,y_train)
#now we will print its classification report and confusion matrix comparing it with the actual values 
print(classification_report(y_test,my_pred))
print(confusion_matrix(y_test,my_pred))


              precision    recall  f1-score   support

           0       0.87      0.52      0.65        25
           1       0.78      0.48      0.60        29
           2       0.77      0.34      0.48        29
           3       0.57      0.27      0.36        30
           4       0.79      0.52      0.62        29
           5       0.55      0.82      0.66        34
           6       0.83      0.17      0.29        29
           7       0.77      0.57      0.65        30
           8       0.95      0.64      0.76        33
           9       1.00      0.71      0.83        24
          10       0.61      0.83      0.70        24
          11       0.49      0.87      0.63        31
          12       0.81      0.33      0.47        39
          13       0.70      0.88      0.78        26
          14       0.58      0.88      0.70        33
          15       0.56      0.71      0.63        28
          16       0.60      0.91      0.72        23
          17       0.38    

In [ ]:
#using  inbuilt multinomial NB 

#import all the required libraries that will be needed throughout the code 
from sklearn import datasets
from sklearn import model_selection
import string
import csv 
import math
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
from sklearn.naive_bayes import MultinomialNB

# a program to fetch all the commonwords from the csv file and store them in a list and then return the list to us.The csv of the common word is attached. Also we have converted all the words to lower case so that case sensitivity is taken care of    

def getcommonwords():
    li=list()
    with open("Commonwords.csv",'r') as obj1:
        obj_data=csv.reader(obj1)
        for i in obj_data:
            if len(i)!=0:
                k=i[0].split()
                for z in k:
                    li.append(str.lower(z))
    return li
# This fuction is used to make a dictionary of all the words present in training data by filtering out the common wordss and punctuation marks and then return the dictionary 
def get_all_training_words(li):
    df=dict()
    for i in x_train:
        j=(i.translate(str.maketrans('','', string.punctuation)).split())
        for k in j:
            k=str.lower(k)
            if k not in li:
                if k not in df:
                    df[k]=0
    return df
#this fucntion iterates obver the training data and then predicts the frequency of each and every word and its number of occourances class wisee thus at primary level we have the classes as keys and at secondary level we have the words and their frequency in that class 
def get_master(df,li,x_train,y_train):
    master={}
    #we make a master dictionary and then call the commonwords function to get the list of commonwords with us 

    #now we will iterate over each and every training point 
    for i in range(len(x_train)):
        master[y_train[i]]=master.get(y_train[i],{})
        #now we will first look for its class and in taking that class of the dictionary as the key we will iterate and first of all we will convert all the sentences into words and remove out all the punctuations from them 
        j=(x_train[i].translate(str.maketrans('','', string.punctuation)).split())
        #now for each and every word we will check that if that word is not a part of common words we will then add it for that particular class and increase the frequency accordingly 
        for k in j:
            k=str.lower(k)
            if k not in li:
                master[y_train[i]][k]=master[y_train[i]].get(k,0)+1
    master2={}
    #now we will filter out the words based on a certain lower cut which means that words below that frequency would be ignored 
    for i in master:
        master2[i]=master2.get(i,{})
        for j in master[i]:
            if master[i][j]>5:
                master2[i][j]=master2[i].get(j,master[i][j])
                

    #now we will return the filtered dictionary and the commonwords list along with that as well 
    return master2
#now the data filter function converts the data in the form so that it could be send to the training algorithm 
def data_fitter(master2,li,df):
    x_li=list()
    y_li=list()
    #we will iterate over each class in master2 dictionary and then we will check for the word from df dictionary if the word is present in that class we will append its frequency else we will append 0 
    for i in master2:
        l2=list()
        for k in df:
                if k in master2[i]:
                    l2.append(master2[i][k])
                else:
                    l2.append(0)
        #now we will append the list that contains freq of all words in order and the value of y for it into 2 separate list and return them 
        x_li.append(l2)
        y_li.append(i)
    return x_li,y_li

#the test converter converts all the testing data into the dictionary with frequency of all the words after filtering out all the punctuations and common words 
def test_converter(li):
    test_val=list()
    for i in range(len(x_test)):
        d=dict()
        j=(x_test[i].translate(str.maketrans('','', string.punctuation)).split())
        for k in j:
            k=str.lower(k)
            if k not in li:
                d[k]=d.get(k,0)+1
        test_val.append(d)
    return test_val
# this function is used to convert the testing data in form of the list that contain freq of each word in the order 
def test_fitter(df,test_val):
    x_pred=list()
    for i in test_val:
        l2=list()
        # we iterate over every testing data and then will check if its present in our test data or not and append freq accordingly
        for k in df:
            if k in i:
                l2.append(i[k])
            else:
                l2.append(0)
        x_pred.append(l2)
    return x_pred
# we will make call to the 20newsgroup dataset in sklearn and remove the header footer  from it    
news = datasets.fetch_20newsgroups(remove=("headers","footers")) 
x = news.data
y=news.target
x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y, test_size = 0.05, random_state = 5)
#call made to programm to get the list of all common words 
li=getcommonwords()
#to get all the words that we have 
df=get_all_training_words(li)
#to convert all the data and get the dictionary with classes and frequency of all words
master2=get_master(df,li,x_train,y_train)
#to convert the data into the training format 
x_li,y_li=data_fitter(master2,li,df)
#to convert testing data to fformat for dictionay
test_val=test_converter(li)
# to convert test data dictionary fit for predictions 
x_pred=test_fitter(df,test_val)
#to make call to multinomialNB and train algo for it 
clf = MultinomialNB()
clf.fit(x_li,y_li)
#to get the predictions from inbuilt MUltinomial NB 
Y_pred = clf.predict(x_pred)
#now we will print its classification report and confusion matrix comparing it with the actual values 
print(classification_report(y_test,Y_pred))
print(confusion_matrix(y_test,Y_pred))

In [3]:
# now we will compare the values of inbuilt multinomialNB with the values we predicted using our own function and print its classification and confusion matrix 
print(classification_report(Y_pred,my_pred))
print(confusion_matrix(Y_pred,my_pred))

              precision    recall  f1-score   support

           0       0.93      0.56      0.70        25
           1       0.78      0.52      0.62        27
           2       0.77      0.53      0.62        19
           3       0.86      0.39      0.53        31
           4       0.84      0.62      0.71        26
           5       0.67      0.72      0.69        47
           6       1.00      0.29      0.44        21
           7       0.91      0.67      0.77        30
           8       0.91      0.74      0.82        27
           9       1.00      0.71      0.83        24
          10       0.64      0.88      0.74        24
          11       0.58      0.78      0.67        41
          12       0.81      0.52      0.63        25
          13       0.73      0.96      0.83        25
          14       0.70      0.95      0.80        37
          15       0.78      0.65      0.71        43
          16       0.77      0.84      0.81        32
          17       0.41    